In [24]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]

,pickup_datetime,pickup_location_id
1000,2023-01-01 15:40:44,HB401
1001,2023-01-01 15:41:36,JC035
1002,2023-01-01 15:41:48,JC035
1003,2023-01-01 15:42:35,JC109
1004,2023-01-01 15:43:38,HB202
1005,2023-01-01 15:43:52,HB202
1006,2023-01-01 15:44:47,HB101
1007,2023-01-01 15:44:53,HB601
1008,2023-01-01 15:45:19,HB601
1009,2023-01-01 15:46:24,JC115


In [3]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56014 entries, 0 to 56013
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_datetime     56014 non-null  datetime64[ns]
 1   pickup_location_id  56014 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 875.3+ KB


In [4]:
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor('h')
rides.tail()

,pickup_datetime,pickup_location_id,pickup_hour
56009,2023-01-31 23:54:41,JC115,2023-01-31 23:00:00
56010,2023-01-31 23:55:39,JC115,2023-01-31 23:00:00
56011,2023-01-31 23:56:37,HB409,2023-01-31 23:00:00
56012,2023-01-31 23:57:41,JC003,2023-01-31 23:00:00
56013,2023-01-31 23:58:11,JC003,2023-01-31 23:00:00


In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [6]:
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.tail()

,pickup_hour,pickup_location_id,rides
25203,2023-01-31 23:00:00,JC023,1
25204,2023-01-31 23:00:00,JC052,1
25205,2023-01-31 23:00:00,JC082,1
25206,2023-01-31 23:00:00,JC109,1
25207,2023-01-31 23:00:00,JC115,6


In [7]:
# Original dataset
data = {
    "hour": ["2025-01-28 08:00", "2025-01-28 08:00", "2025-01-28 09:00", "2025-01-28 10:00"],
    "location_id": [1, 2, 1, 2],
    "rides": [10, 5, 8, 12]
}
df = pd.DataFrame(data)
df["hour"] = pd.to_datetime(df["hour"])

df

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10
1,2025-01-28 08:00:00,2,5
2,2025-01-28 09:00:00,1,8
3,2025-01-28 10:00:00,2,12


In [8]:
# get all the hours
hour_col = "hour"
full_hours = pd.date_range(
    start=df[hour_col].min(),
    end=df[hour_col].max(),
    freq="h"
)

full_hours

DatetimeIndex(['2025-01-28 08:00:00', '2025-01-28 09:00:00',
               '2025-01-28 10:00:00'],
              dtype='datetime64[ns]', freq='H')

In [9]:
# unique location ids
location_col = "location_id"
all_locations = df[location_col].unique()
all_locations

array([1, 2])

In [10]:
# Create a list to store the complete data
complete_data = []
rides_col = "rides"
# Iterate over all combinations of hours and locations
for hour in full_hours:
    for location in all_locations:
        # Check if the combination exists in the original DataFrame
        subset = df[(df[hour_col] == hour) & (df[location_col] == location)]
        if not subset.empty:
            # If the combination exists, append the row
            complete_data.append(subset.iloc[0].to_dict())
        else:
            # If the combination is missing, append a row with 0 rides
            complete_data.append({hour_col: hour, location_col: location, rides_col: 0})

df_complete = pd.DataFrame(complete_data)
df_complete

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10
1,2025-01-28 08:00:00,2,5
2,2025-01-28 09:00:00,1,8
3,2025-01-28 09:00:00,2,0
4,2025-01-28 10:00:00,1,0
5,2025-01-28 10:00:00,2,12


In [11]:
df_complete.sort_values(["location_id", "hour"])

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10
2,2025-01-28 09:00:00,1,8
4,2025-01-28 10:00:00,1,0
1,2025-01-28 08:00:00,2,5
3,2025-01-28 09:00:00,2,0
5,2025-01-28 10:00:00,2,12


In [12]:
# Create full combinations of hours and locations
full_combinations = pd.DataFrame(
    [(hour, location) for hour in full_hours for location in all_locations],
    columns=["hour", "location_id"]
)

full_combinations

,hour,location_id
0,2025-01-28 08:00:00,1
1,2025-01-28 08:00:00,2
2,2025-01-28 09:00:00,1
3,2025-01-28 09:00:00,2
4,2025-01-28 10:00:00,1
5,2025-01-28 10:00:00,2


In [13]:
df

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10
1,2025-01-28 08:00:00,2,5
2,2025-01-28 09:00:00,1,8
3,2025-01-28 10:00:00,2,12


In [14]:
# Merge with the original dataset
merged_df = pd.merge(full_combinations, df, on=["hour", "location_id"], how="left")
merged_df

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10.0
1,2025-01-28 08:00:00,2,5.0
2,2025-01-28 09:00:00,1,8.0
3,2025-01-28 09:00:00,2,NaN
4,2025-01-28 10:00:00,1,NaN
5,2025-01-28 10:00:00,2,12.0


In [15]:
merged_df["rides"] = merged_df["rides"].fillna(0).astype(int)
merged_df

,hour,location_id,rides
0,2025-01-28 08:00:00,1,10
1,2025-01-28 08:00:00,2,5
2,2025-01-28 09:00:00,1,8
3,2025-01-28 09:00:00,2,0
4,2025-01-28 10:00:00,1,0
5,2025-01-28 10:00:00,2,12


In [16]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [17]:
agg_rides

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB101,1
1,2023-01-01 00:00:00,HB102,2
2,2023-01-01 00:00:00,HB202,2
3,2023-01-01 00:00:00,HB303,1
4,2023-01-01 00:00:00,HB401,1
...,...,...,...
25203,2023-01-31 23:00:00,JC023,1
25204,2023-01-31 23:00:00,JC052,1
25205,2023-01-31 23:00:00,JC082,1
25206,2023-01-31 23:00:00,JC109,1


In [18]:
hour_col = "pickup_hour"
location_col = "pickup_location_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

In [19]:
pd.set_option('display.max_rows', 2000)
agg_data_filled.head(20)


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,5532.01,0
1,2023-01-01 01:00:00,5532.01,0
2,2023-01-01 02:00:00,5532.01,0
3,2023-01-01 03:00:00,5532.01,0
4,2023-01-01 04:00:00,5532.01,0
5,2023-01-01 05:00:00,5532.01,0
6,2023-01-01 06:00:00,5532.01,0
7,2023-01-01 07:00:00,5532.01,0
8,2023-01-01 08:00:00,5532.01,0
9,2023-01-01 09:00:00,5532.01,0


In [20]:
rides.pickup_location_id.isin(['HB101'])

0        False
1        False
2        False
3        False
4        False
         ...  
56009    False
56010    False
56011    False
56012    False
56013    False
Name: pickup_location_id, Length: 56014, dtype: bool

In [21]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none"
    )

    fig.show()

In [22]:
plot_rides(agg_data_filled, locations=['HB101', 'HB102'])

In [23]:
month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

agg_data_filled.to_parquet(path)